In [ ]:
import os
import re
import cv2
import random
import numpy as np
import pandas as pd
from time import time
from tqdm import tqdm
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNEL = 3
NUM_CLASSES = 32
BATCH_SIZE = 32
DROPOUT = 0.3

IMG_PATH = r"../input/camvid/CamVid/train/"
MASK_PATH = r"../input/camvid/CamVid/train_labels/"

strategy = tf.distribute.get_strategy()

seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
def LoadImage(name, path, seg_path):
    img = Image.open(os.path.join(path, name))
    
    name_seg = re.sub('\.', '_L.', name)
    
    img = np.array(Image.open(os.path.join(path, name)).convert('RGB').resize((IMG_HEIGHT, IMG_WIDTH), Image.ANTIALIAS))
    mask = np.array(Image.open(os.path.join(seg_path, name_seg)).convert('RGB').resize((IMG_HEIGHT, IMG_WIDTH), Image.ANTIALIAS))

    return img/255.0, mask/255.0

In [ ]:
from sklearn.cluster import KMeans

files =os.listdir(IMG_PATH)[0:10]
colors = []
for file in files:
    img, seg = LoadImage(file, IMG_PATH, MASK_PATH)
    colors.append(seg.reshape(seg.shape[0]*seg.shape[1], 3))
colors = np.array(colors)
colors = colors.reshape((colors.shape[0]*colors.shape[1],3))

km = KMeans(32)
km.fit(colors)

In [ ]:
import seaborn as sns 

def LayersToRGBImage(img):
    
    colors = [(234,222,218), (153,136,136), (100,44,169),
             (23,126,137), (8,96,95), (161,205,244),
             (124,128,155), (61,59,60),(127,121,121),
             (53,45,57), (255,105,120), (255,255,130),
             (193,189,179), (95,91,107), (217,187,249), (170,159,177),
             (230,228,206), (122,48,108), (3,181,170),
             (78,82,131), (244,172,50),(177,145,255),
             (82,21,78), (17,19,68), (218,224,242),
             (84,19,136), (255,212,0), (96,0,71),(217,247,250),
             (202,240,248), (3,113,113), (255,87,159),
             (212,170,125), (0,0,0)]
    
    nimg = np.zeros((img.shape[0], img.shape[1], 3))
    for i in range(img.shape[2]):
        c = img[:,:,i]
        col = colors[i]
        
        for j in range(3):
            nimg[:,:,j]+=col[j]*c
    nimg = nimg/255.0
    return nimg

In [ ]:
def ColorsToClass(seg):
    s = seg.reshape((seg.shape[0]*seg.shape[1],3))
    s = km.predict(s)
    s = s.reshape((seg.shape[0], seg.shape[1]))
    
    n = len(km.cluster_centers_)
    
    cls = np.zeros((seg.shape[0], seg.shape[1], n))
    
    for i in range(n):
        m = np.copy(s)
        m[m!=i] = 0
        m[m!=0] = 1
        
        cls[:,:,i]=m
        
    return cls

img, seg = LoadImage(os.listdir(IMG_PATH)[1], IMG_PATH, MASK_PATH)
seg2 = ColorsToClass(seg)
seg2 = LayersToRGBImage(seg2)
total = cv2.addWeighted(img, 0.6, seg2, 0.4, 0)
plt.imshow(total[:,:,:])
plt.show()

In [ ]:
def Generate(path=IMG_PATH, seg_path=MASK_PATH, batch_size=BATCH_SIZE):
    
    files = os.listdir(path)
    while True:
        imgs=[]
        segs=[]
        
        for i in range(batch_size):
            file = random.sample(files,1)[0]
   
            img, seg = LoadImage(file, path, seg_path)
            
            seg = ColorsToClass(seg)
            
            imgs.append(img)
            segs.append(seg)
        yield np.array(imgs), np.array(segs)
        
gen = Generate()
imgs, segs = next(gen)

plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(imgs[0])
plt.subplot(122)
plt.imshow(LayersToRGBImage(segs[0]))
plt.show()

In [ ]:
def block(x, n_convs, filters, kernel_size, activation, pool_size, pool_stride, block_name):

    for i in range(n_convs):
        x = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, padding='same', name="{}_conv{}".format(block_name, i + 1))(x)
    
    x = tf.keras.layers.MaxPooling2D(pool_size=pool_size, strides=pool_stride, name="{}_pool{}".format(block_name, i+1))(x)

    return x

In [ ]:
# download the weights
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

# assign to a variable
vgg_weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
def VGG_16(image_input):

    # create 5 blocks with increasing filters at each stage. 
    x = block(image_input, n_convs=2, filters=64, kernel_size=(3, 3), activation='relu', pool_size=(2, 2), pool_stride=(2, 2), block_name='block1')
    p1 = x

    x = block(x, n_convs=2, filters=128, kernel_size=(3, 3), activation='relu', pool_size=(2, 2), pool_stride=(2, 2), block_name='block2')
    p2 = x 

    x = block(x, n_convs=3, filters=256, kernel_size=(3, 3), activation='relu', pool_size=(2, 2), pool_stride=(2, 2), block_name='block3')
    p3 = x

    x = block(x, n_convs=3, filters=512, kernel_size=(3, 3), activation='relu', pool_size=(2, 2), pool_stride=(2, 2), block_name='block4')
    p4 = x

    x = block(x, n_convs=3, filters=512, kernel_size=(3, 3), activation='relu', pool_size=(2, 2), pool_stride=(2, 2), block_name='block5')
    p5 = x

    # create the vgg model
    vgg = tf.keras.Model(image_input, p5)

    n = 4096

    # our input images are 224x224 pixels so they will be downsampled to 7x7 after the pooling layers above.
    # we can extract more features by chaining two more convolution layers.

    c6 = tf.keras.layers.Conv2D(n, (7, 7), activation='relu', padding='same', name='conv6')(p5)
    c7 = tf.keras.layers.Conv2D(n, (1, 1), activation='relu', padding='same', name='conv7')(c6)

    return (p1, p2, p3, p4, c7)

In [ ]:
def fcn8_decoder(convs, n_classes):

    # unpack the output of the encoder
    f1, f2, f3, f4, f5 = convs
  
    # upsample the output of the encoder then crop extra pixels that were introduced
    o = tf.keras.layers.Conv2DTranspose(n_classes , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False )(f5)
    o = tf.keras.layers.Cropping2D(cropping=(1,1))(o)

    # load the pool 4 prediction and do a 1x1 convolution to reshape it to the same shape of `o` above
    o2 = f4
    o2 = ( tf.keras.layers.Conv2D(n_classes , ( 1 , 1 ) , activation='relu' , padding='same'))(o2)

    # add the results of the upsampling and pool 4 prediction
    o = tf.keras.layers.Add()([o, o2])

    # upsample the resulting tensor of the operation you just did
    o = (tf.keras.layers.Conv2DTranspose( n_classes , kernel_size=(4,4) ,  strides=(2,2) , use_bias=False ))(o)
    o = tf.keras.layers.Cropping2D(cropping=(1, 1))(o)

    # load the pool 3 prediction and do a 1x1 convolution to reshape it to the same shape of `o` above
    o2 = f3
    o2 = ( tf.keras.layers.Conv2D(n_classes , ( 1 , 1 ) , activation='relu' , padding='same'))(o2)

    # add the results of the upsampling and pool 3 prediction
    o = tf.keras.layers.Add()([o, o2])
  
    # upsample up to the size of the original image
    o = tf.keras.layers.Conv2DTranspose(n_classes , kernel_size=(8,8) ,  strides=(8,8) , use_bias=False )(o)

    # append a softmax to get the class probabilities
    o = (tf.keras.layers.Activation('softmax'))(o)

    return o

In [ ]:
def segmentation_model():
  
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL))
    convs = VGG_16(image_input=inputs)
    outputs = fcn8_decoder(convs, 32)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
  
    return model

In [ ]:
model = segmentation_model()
model.summary()

In [ ]:
train_gen = Generate()
val_gen = Generate(path="../input/camvid/CamVid/val/", seg_path="../input/camvid/CamVid/val_labels/")

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD

adam = Adam(lr=0.001, decay=1e-06)
sgd = SGD(lr=1E-3, momentum=0.9, nesterov=True)

filepath = "best-model-vgg.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=7, verbose=1,
    mode='min', restore_best_weights=True
)

reduceLR = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, mode='max', min_lr=1e-15)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
clb = [reduceLR, checkpoint, early_stop]

In [ ]:
batch_size = BATCH_SIZE
num_of_training_samples = len(os.listdir(IMG_PATH))
num_of_testing_samples = len(os.listdir("../input/camvid/CamVid/val/"))

history = model.fit_generator(train_gen, epochs=200, steps_per_epoch=num_of_training_samples//batch_size,
                       validation_data=val_gen, validation_steps=num_of_testing_samples//batch_size, callbacks=[clb])

In [ ]:
model.load_weights("best-model-vgg.hdf5")

In [ ]:
loss = history.history["val_loss"]
acc = history.history["val_accuracy"] #accuracy

plt.figure(figsize=(12, 6))
plt.subplot(211)
plt.title("Val. Loss")
plt.plot(loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(212)
plt.title("Val. Accuracy")
plt.plot(acc)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.savefig("learn.png", dpi=150)
plt.show()

In [ ]:
def give_color_to_seg_img(seg, n_classes=NUM_CLASSES):
    
    seg_img = np.zeros( (seg.shape[0],seg.shape[1],3) ).astype('float')
    colors = sns.color_palette("hls", n_classes)
    
    for c in range(n_classes):
        segc = (seg == c)
        seg_img[:,:,0] += (segc*( colors[c][0] ))
        seg_img[:,:,1] += (segc*( colors[c][1] ))
        seg_img[:,:,2] += (segc*( colors[c][2] ))

    return(seg_img)

In [ ]:
#val_gen = DataGenerator(valid_folder)
max_show = 1
imgs, segs = next(val_gen)
pred = model.predict(imgs)

for i in range(max_show):
    _p = give_color_to_seg_img(np.argmax(pred[i], axis=-1))
    _s = give_color_to_seg_img(np.argmax(segs[i], axis=-1))

    predimg = cv2.addWeighted(imgs[i]/255, 0.5, _p, 0.5, 0)
    trueimg = cv2.addWeighted(imgs[i]/255, 0.5, _s, 0.5, 0)
    
    plt.figure(figsize=(12,6))
    plt.subplot(121)
    plt.title("Prediction")
    plt.imshow(predimg)
    plt.axis("off")
    plt.subplot(122)
    plt.title("Original")
    plt.imshow(trueimg)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig("pred_"+str(i)+".png", dpi=150)
    plt.show()